In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

22/11/18 07:26:30 WARN Utils: Your hostname, potato resolves to a loopback address: 127.0.1.1; using 192.168.43.253 instead (on interface wlp3s0)
22/11/18 07:26:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 07:26:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("data/01_2022.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- _id: integer (nullable = true)
 |-- SiteId: integer (nullable = true)
 |-- Site_Name: string (nullable = true)
 |-- Site_Brand: string (nullable = true)
 |-- Sites_Address_Line_1: string (nullable = true)
 |-- Site_Suburb: string (nullable = true)
 |-- Site_State: string (nullable = true)
 |-- Site_Post_Code: integer (nullable = true)
 |-- Site_Latitude: double (nullable = true)
 |-- Site_Longitude: double (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- TransactionDateutc: timestamp (nullable = true)



In [3]:
df.show(5)

+---+--------+------------------+-----------+--------------------+-----------+----------+--------------+-------------+--------------+-----------+-----+-------------------+
|_id|  SiteId|         Site_Name| Site_Brand|Sites_Address_Line_1|Site_Suburb|Site_State|Site_Post_Code|Site_Latitude|Site_Longitude|  Fuel_Type|Price| TransactionDateutc|
+---+--------+------------------+-----------+--------------------+-----------+----------+--------------+-------------+--------------+-----------+-----+-------------------+
|  1|61290151|   Surat 24/7 Fuel|Independent|  61 Burrowes Street|      SURAT|       QLD|          4417|   -27.151687|    149.067742|     Diesel| 1659|2022-01-20 02:56:00|
|  2|61290151|   Surat 24/7 Fuel|Independent|  61 Burrowes Street|      SURAT|       QLD|          4417|   -27.151687|    149.067742|PULP 98 RON| 1859|2022-01-20 02:56:00|
|  3|61290151|   Surat 24/7 Fuel|Independent|  61 Burrowes Street|      SURAT|       QLD|          4417|   -27.151687|    149.067742|   Unle

In [4]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [6]:
from pyspark.sql.functions import col
df.where(col("SiteId") != 61290151)\
.select("Price", "Fuel_Type", "Site_Name", "Site_Brand")\
.show(5, False)

+-----+---------+------------------+----------+
|Price|Fuel_Type|Site_Name         |Site_Brand|
+-----+---------+------------------+----------+
|1680 |Diesel   |Lowes Mungindi Opt|BP        |
|1710 |Unleaded |Lowes Mungindi Opt|BP        |
|1584 |e10      |7-Eleven Coomera  |7 Eleven  |
|1579 |e10      |7-Eleven Coomera  |7 Eleven  |
|1879 |e10      |7-Eleven Coomera  |7 Eleven  |
+-----+---------+------------------+----------+
only showing top 5 rows



In [7]:
from pyspark.sql.functions import col
df.where(col("SiteId") != 61290151)\
.select("Price", "Fuel_Type", "Site_Name", "Site_Brand")\
.show(5, True)

+-----+---------+------------------+----------+
|Price|Fuel_Type|         Site_Name|Site_Brand|
+-----+---------+------------------+----------+
| 1680|   Diesel|Lowes Mungindi Opt|        BP|
| 1710| Unleaded|Lowes Mungindi Opt|        BP|
| 1584|      e10|  7-Eleven Coomera|  7 Eleven|
| 1579|      e10|  7-Eleven Coomera|  7 Eleven|
| 1879|      e10|  7-Eleven Coomera|  7 Eleven|
+-----+---------+------------------+----------+
only showing top 5 rows



In [8]:
df.where("SiteId = 61290151").show(10, False)
df.where("SiteId <> 61290151").show(10, False)

+---+--------+---------------+-----------+--------------------+-----------+----------+--------------+-------------+--------------+-----------+-----+-------------------+
|_id|SiteId  |Site_Name      |Site_Brand |Sites_Address_Line_1|Site_Suburb|Site_State|Site_Post_Code|Site_Latitude|Site_Longitude|Fuel_Type  |Price|TransactionDateutc |
+---+--------+---------------+-----------+--------------------+-----------+----------+--------------+-------------+--------------+-----------+-----+-------------------+
|1  |61290151|Surat 24/7 Fuel|Independent|61 Burrowes Street  |SURAT      |QLD       |4417          |-27.151687   |149.067742    |Diesel     |1659 |2022-01-20 02:56:00|
|2  |61290151|Surat 24/7 Fuel|Independent|61 Burrowes Street  |SURAT      |QLD       |4417          |-27.151687   |149.067742    |PULP 98 RON|1859 |2022-01-20 02:56:00|
|3  |61290151|Surat 24/7 Fuel|Independent|61 Burrowes Street  |SURAT      |QLD       |4417          |-27.151687   |149.067742    |Unleaded   |1659 |2022-01

In [17]:
df.where("Sites_Address_Line_1 like 'Pacific%'").show(10, True)

+---+--------+----------------+----------+--------------------+-----------+----------+--------------+-------------+--------------+---------+-----+-------------------+
|_id|  SiteId|       Site_Name|Site_Brand|Sites_Address_Line_1|Site_Suburb|Site_State|Site_Post_Code|Site_Latitude|Site_Longitude|Fuel_Type|Price| TransactionDateutc|
+---+--------+----------------+----------+--------------------+-----------+----------+--------------+-------------+--------------+---------+-----+-------------------+
|  6|61401007|7-Eleven Coomera|  7 Eleven|     Pacific Highway|    Coomera|       QLD|          4209|   -27.868591|    153.314206|      e10| 1584|2021-12-31 18:20:00|
|  7|61401007|7-Eleven Coomera|  7 Eleven|     Pacific Highway|    Coomera|       QLD|          4209|   -27.868591|    153.314206|      e10| 1579|2022-01-03 23:15:00|
|  8|61401007|7-Eleven Coomera|  7 Eleven|     Pacific Highway|    Coomera|       QLD|          4209|   -27.868591|    153.314206|      e10| 1879|2022-01-07 00:40:00

In [19]:
from pyspark.sql.functions import instr
Site_State = col("Site_State") == "QLD"
priceFilter = col("Price") > 1600
df.withColumn("isExpensive", Site_State & priceFilter)\
.where("isExpensive")\
.select("Price", "isExpensive").show(5)

+-----+-----------+
|Price|isExpensive|
+-----+-----------+
| 1659|       true|
| 1859|       true|
| 1659|       true|
| 1680|       true|
| 1710|       true|
+-----+-----------+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import min, max
df.select(min("Price"), max("Price")).show()

+----------+----------+
|min(Price)|max(Price)|
+----------+----------+
|       149|      9999|
+----------+----------+

